In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from statistics import *
pd.options.display.max_rows = 450

In [2]:
TestPW = pd.read_csv("TestWaves.csv", header=None)
Sep = pd.read_csv("PWVSeparate.csv", header=0)
patPWV = pd.read_csv("PWVbyPatient.csv", header=None)

In [3]:
#change pat variable to specify patient [1,20]
pat = 20
#x = number of usable waves for patient
x = Sep.iloc[:,pat].last_valid_index() + 1
print(x)

pp_pres = []
avg_sys_rise = []
sys_rise_area = []
t_sys_rise = []
avg_dec = []
t_dec = []
dec_area = []
avg_sys = []
slope_sys = []
sys_area = []
t_sys = []
avg_sys_dec = []
dn_sys = []
sys_dec_area = []
t_sys_dec = []
avg_sys_dec_nodia = []
avg_sys_nodia = []
avg_sys_rise_nodia = []
avg_dec_nodia = []
slope_dia = []
t_dia = []
avg_dia = []
dn_dia = []




metrics = []

5


In [4]:
for i in range(0,x):
    
    #calculate metrics for each indexed wave saved in individual metric arrays
    #remove outliers, save only mean for each metric
    #append metric means to new df
    
    #defining wave based on lower bound / upper bounds of handpicked, usable waves
    col = Sep.iloc[:x,pat]
    split = col[i].split("-")
    lowerB = int(split[0])
    upperB = int(split[1])
    
    #pulling wave using bounds
    wave = patPWV.iloc[lowerB:upperB,pat].reset_index(drop=True)
    
    #finding maximum, defining region for dic notch
    sysMaxi = wave.idxmax()
    reg = wave[round(sysMaxi*1.05):round(sysMaxi+(len(wave)*.28))]
    diff = reg.diff()
    
    NP = [0,0]

    
    #find dic notch
    ser = [0,0]
    counter = 0
    for index,value in diff.items():
        if value < 1:
            counter = counter + 1
        if value >= 0:
            if counter > 0 and counter > ser[1]:
                ser[0] = index
                ser[1] = counter
            counter = 0
    
    

    
    #if no dic notch found:
    if ser[0] == 0 or ser[1] < len(reg)*.05:
        #estimate diastolic pressure as flattest point -> highest diff
        #range for diastolic pressure:
        diaReg = diff.tail(n=round(len(diff)*.9))
        diaP = diaReg.idxmax()
        dicN = round(mean([diaReg.idxmax(),diaReg.idxmin()]))
        #print("NO DIC NOTCH FOUND, estimating...")
    else:
        dicN = ser[0]
        if round(2*sysMaxi) > dicN+1:
            diaP = wave[dicN+1:round(2*sysMaxi)].idxmax()
        else:
            diaP = dicN+1
    
    NP[0] = dicN
    NP[1] = diaP
    
    
        
    #plt.plot(wave)
    #plt.axvspan(round(sysMaxi*1.05), round(sysMaxi+(len(wave)*.28)), color='red', alpha=0.5)
    #plt.axvline(x=NP[0])
    #plt.show()

    #saving dic notch and diastolic pressure for functions
    dicNotch = NP[0]
    diaP = NP[1]
    
    #beginning and end of wave
    beg = 0
    end = wave.last_valid_index()
    sysMaxi = wave.idxmax()
    

    #add calculated metrics to lists
    pp_pres.append(np.sum(wave))
    avg_sys_rise.append(wave[beg:sysMaxi].mean())
    sys_rise_area.append(sum(wave[beg:sysMaxi]))
    t_sys_rise.append(sysMaxi)
    avg_dec.append(wave[sysMaxi:end].mean())
    t_dec.append(end - sysMaxi)
    dec_area.append(np.sum(wave[sysMaxi:end]))
    avg_sys.append(wave[beg:dicNotch].mean())
    slope_sys.append((wave[dicNotch] - wave[beg]) / dicNotch)
    sys_area.append(sum(wave[beg:dicNotch]))
    t_sys.append(dicNotch)
    avg_sys_dec.append(wave[sysMaxi:dicNotch].mean())
    dn_sys.append(wave[sysMaxi] - wave[dicNotch])
    sys_dec_area.append(np.sum(wave[sysMaxi:dicNotch]))
    t_sys_dec.append(dicNotch - sysMaxi)
    avg_sys_dec_nodia.append(avg_sys_dec[i] - wave[diaP])
    avg_sys_nodia.append(avg_sys[i] - wave[diaP])
    avg_sys_rise_nodia.append(avg_sys_rise[i] - wave[diaP])
    avg_dec_nodia.append(avg_dec[i] - wave[diaP])
    slope_dia.append((wave[end] - wave[dicNotch]) / (end - dicNotch))
    t_dia.append(end - dicNotch)
    avg_dia.append(wave[dicNotch:end].mean())
    dn_dia.append(wave[diaP] - wave[dicNotch])


#remove outliers and calculate mean per metic in each list    
    
pp_pres = pd.Series(pp_pres)
pp_pres = pp_pres[pp_pres.between(pp_pres.quantile(.15), pp_pres.quantile(.85))]
pp_pres = mean(pp_pres)

avg_sys_rise = pd.Series(avg_sys_rise)
avg_sys_rise = avg_sys_rise[avg_sys_rise.between(avg_sys_rise.quantile(.15), avg_sys_rise.quantile(.85))]
avg_sys_rise = mean(avg_sys_rise)

sys_rise_area = pd.Series(sys_rise_area)
sys_rise_area = sys_rise_area[sys_rise_area.between(sys_rise_area.quantile(.15), sys_rise_area.quantile(.85))]
sys_rise_area = mean(sys_rise_area)

t_sys_rise = pd.Series(t_sys_rise)
t_sys_rise = t_sys_rise[t_sys_rise.between(t_sys_rise.quantile(.15), t_sys_rise.quantile(.85))]
t_sys_rise = mean(t_sys_rise)

avg_dec = pd.Series(avg_dec)
avg_dec = avg_dec[avg_dec.between(avg_dec.quantile(.15), avg_dec.quantile(.85))]
avg_dec = mean(avg_dec)

t_dec = pd.Series(t_dec)
t_dec = t_dec[t_dec.between(t_dec.quantile(.15), t_dec.quantile(.85))]
t_dec = mean(t_dec)

dec_area = pd.Series(dec_area)
dec_area = dec_area[dec_area.between(dec_area.quantile(.15), dec_area.quantile(.85))]
dec_area = mean(dec_area)

avg_sys = pd.Series(avg_sys)
avg_sys = avg_sys[avg_sys.between(avg_sys.quantile(.15), avg_sys.quantile(.85))]
avg_sys = mean(avg_sys)

slope_sys = pd.Series(slope_sys)
slope_sys = slope_sys[slope_sys.between(slope_sys.quantile(.15), slope_sys.quantile(.85))]
slope_sys = mean(slope_sys)

sys_area = pd.Series(sys_area)
sys_area = sys_area[sys_area.between(sys_area.quantile(.15), sys_area.quantile(.85))]
sys_area = mean(sys_area)

t_sys = pd.Series(t_sys)
t_sys = t_sys[t_sys.between(t_sys.quantile(.15), t_sys.quantile(.85))]
t_sys = mean(t_sys)

avg_sys_dec = pd.Series(avg_sys_dec)
avg_sys_dec = avg_sys_dec[avg_sys_dec.between(avg_sys_dec.quantile(.15), avg_sys_dec.quantile(.85))]
avg_sys_dec = mean(avg_sys_dec)

dn_sys = pd.Series(dn_sys)
dn_sys = dn_sys[dn_sys.between(dn_sys.quantile(.15), dn_sys.quantile(.85))]
dn_sys = mean(dn_sys)

sys_dec_area = pd.Series(sys_dec_area)
sys_dec_area = sys_dec_area[sys_dec_area.between(sys_dec_area.quantile(.15), sys_dec_area.quantile(.85))]
sys_dec_area = mean(sys_dec_area)

t_sys_dec = pd.Series(t_sys_dec)
t_sys_dec = t_sys_dec[t_sys_dec.between(t_sys_dec.quantile(.15), t_sys_dec.quantile(.85))]
t_sys_dec = mean(t_sys_dec)

avg_sys_dec_nodia = pd.Series(avg_sys_dec_nodia)
avg_sys_dec_nodia = avg_sys_dec_nodia[avg_sys_dec_nodia.between(avg_sys_dec_nodia.quantile(.15), avg_sys_dec_nodia.quantile(.85))]
avg_sys_dec_nodia = mean(avg_sys_dec_nodia)

avg_sys_nodia = pd.Series(avg_sys_nodia)
avg_sys_nodia = avg_sys_nodia[avg_sys_nodia.between(avg_sys_nodia.quantile(.15), avg_sys_nodia.quantile(.85))]
avg_sys_nodia = mean(avg_sys_nodia)

avg_sys_rise_nodia = pd.Series(avg_sys_rise_nodia)
avg_sys_rise_nodia = avg_sys_rise_nodia[avg_sys_rise_nodia.between(avg_sys_rise_nodia.quantile(.15), avg_sys_rise_nodia.quantile(.85))]
avg_sys_rise_nodia = mean(avg_sys_rise_nodia)

avg_dec_nodia = pd.Series(avg_dec_nodia)
avg_dec_nodia = avg_dec_nodia[avg_dec_nodia.between(avg_dec_nodia.quantile(.15), avg_dec_nodia.quantile(.85))]
avg_dec_nodia = mean(avg_dec_nodia)
             
slope_dia = pd.Series(slope_dia)
slope_dia = slope_dia[slope_dia.between(slope_dia.quantile(.15), slope_dia.quantile(.85))]
slope_dia = mean(slope_dia)

t_dia = pd.Series(t_dia)
t_dia = t_dia[t_dia.between(t_dia.quantile(.15), t_dia.quantile(.85))]
t_dia = mean(t_dia)

avg_dia = pd.Series(avg_dia)
avg_dia = avg_dia[avg_dia.between(avg_dia.quantile(.15), avg_dia.quantile(.85))]
avg_dia = mean(avg_dia)

dn_dia = pd.Series(dn_dia)
dn_dia = dn_dia[dn_dia.between(dn_dia.quantile(.15), dn_dia.quantile(.85))]
dn_dia = mean(dn_dia)









#add metrics to big data frame

metrics.append([pp_pres,avg_sys_rise,sys_rise_area,t_sys_rise,avg_dec,t_dec,dec_area,avg_sys,slope_sys,sys_area,t_sys,avg_sys_dec,dn_sys,sys_dec_area,t_sys_dec,avg_sys_dec_nodia,avg_sys_nodia,avg_sys_rise_nodia,avg_dec_nodia,slope_dia,t_dia,avg_dia,dn_dia,avg_sys_nodia])
metrics

[[12398.466666666665,
  74.93875438596491,
  2572.5333333333333,
  31.5,
  71.63173417875508,
  137,
  9751.066666666666,
  77.10230664328851,
  0.18202629721538235,
  4548.599999999999,
  59,
  80.04786492374727,
  10.149999999999995,
  1962.8,
  24.666666666666668,
  2.485555555555559,
  0.2560607284179923,
  -2.010947368421052,
  -5.75137084407596,
  -0.10120628392367526,
  112.33333333333333,
  69.3928582619887,
  4.0,
  0.2560607284179923]]